In [1]:
import nltk
import re
import pandas as pd
from IPython.display import clear_output
from collections import defaultdict
from string import punctuation

#stopwords em portugues
from nltk.corpus import stopwords
stopwords = set(stopwords.words('portuguese') + list(punctuation) + ['“','”'])

#funçao pra limpeza de texto
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=False)

In [2]:
#funçao pra leitura de dados e criaçao de dataframe
def json_to_dataframe(nome_json, colunas, chunk):
    dados = pd.read_json(nome_json, convert_dates = False, lines = True, chunksize = chunk)    
    
    for tweets_retweets in dados:
        yield tweets_retweets[colunas]

In [3]:
#funçao pra limpar texto e retirar stopwords
def tokenizar_texto(texto_tweet):
    texto_tokenizado = tweet_tokenizer.tokenize(texto_tweet)

    if texto_tokenizado[0] == "rt" and texto_tokenizado[1] == ":":
        del texto_tokenizado[0]
    
    palavras_sem_stopwords = [palavra for palavra in texto_tokenizado if palavra not in stopwords] 
    return palavras_sem_stopwords

In [10]:
#funçao pra criar dicionario com a data e quantidade dos termos
def contagem_termos_data(termos_lista, gerador):
    
    dict_datas_termos = defaultdict(dict)
    contador = 0
    
    for dataframe in gerador:        
        for linha in dataframe.values:            
            texto_tweet = list(tokenizar_texto(linha[0]))

            for termo in texto_tweet:                
                if termo in termos_lista:
                    data = linha[1]['$date'][:10]
                    if data not in dict_datas_termos or termo not in dict_datas_termos[data]:
                        dict_datas_termos[data][termo] = 0
                    dict_datas_termos[data][termo] += 1
            clear_output()
            contador += 1        
            print("%s tweets processados" %(contador))            
            print(dict(dict_datas_termos))
            
    escrever_arq_csv(dict_datas_termos)

In [13]:
#funçao pra escrever os resultados no arquivo csv
def escrever_arq_csv(dict_datas_termos):
    with open('termos_economicos_100k.csv', 'w', encoding="utf-8") as f:
        f.write("Data;Termo;Frequencia\n")

        for key in dict_datas_termos.keys():
            for termo in dict_datas_termos[key].keys():
                f.write("%s;%s;%s\n"%(key, termo, dict_datas_termos[key][termo]))

In [14]:
#execuçao do codigo
gerador = json_to_dataframe('100k_origin.json',["text","created_at"], 30)

contagem_termos_data(['incerteza', 'desemprego', 'desempregado', 'desempregada', 'demissão', 'demitido', 'demitida',
                      'auxilio emergencial', 'mei', 'doação', 'informalidade', 'bico', 'camelô'], gerador)

100000 tweets processados
{'2020-03-20': {'demissão': 3, 'demitida': 38, 'doação': 82, 'desempregado': 8, 'mei': 6, 'desemprego': 5, 'incerteza': 5, 'bico': 4, 'desempregada': 14, 'demitido': 2, 'informalidade': 1}}
